In [ ]:
!pip install --upgrade tensorflow==1.15

     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 3.8MB 31.4MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 512kB 41.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=6eee0030743efbf5a0b6b2c66202518c9d5140a7f31650adb4fa943e33106425
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installati

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
dataset = pd.read_csv ('b_depressed.csv')
dataset = dataset.dropna()



> Non-PCA + All Inputs



In [ ]:
x_features = dataset[['sex', 'Age', 'Married', 'Number_children', 'education_level', 'total_members', 'gained_asset', 'durable_asset', 'save_asset', 'living_expenses', 'other_expenses', 'incoming_salary', 'incoming_own_farm', 'incoming_business', 'incoming_no_business', 'incoming_agricultural', 'farm_expenses', 'labor_primary', 'lasting_investment', 'no_lasting_investmen']].values
y_target = dataset[['depressed']].values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split (x_features, y_target, test_size = 0.2, random_state=0)

In [ ]:
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
y_train, y_test = y_train.astype('float32'), y_test.astype('float32')

In [ ]:
layers = {
    'input' : 20,
    'hidden' : 1,
    'output' : 1
}

input_hidden = {
    'weight': tf.Variable(tf.random_normal([layers['input'], layers['hidden']], seed=0)),
    'bias': tf.Variable(tf.random_normal([layers['hidden']], seed=0))
}

hidden_output = {
    'weight': tf.Variable(tf.random_normal([layers['hidden'], layers['output']], seed=0)),
    'bias': tf.Variable(tf.random_normal([layers['output']], seed=0))
}

In [ ]:
def feed_forward(input):
  x1 = tf.matmul(input, input_hidden['weight']) + input_hidden['bias']
  y1 = tf.nn.sigmoid(x1)

  x2 = tf.matmul(y1, hidden_output['weight']) + hidden_output['bias']
  y2 = tf.nn.sigmoid(x2)

  return y2

In [ ]:
input_placeholder = tf.placeholder(tf.float32, [None, layers['input']])
output_placeholder = tf.placeholder(tf.float32, [None, layers['output']])

output = feed_forward(input_placeholder)
error = tf.reduce_mean(0.5 * (output_placeholder - output) ** 2)

learning_rate = 0.2
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

In [ ]:
epoch = 200

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  for i in range(epoch):
    train_dict = {
        input_placeholder: x_train,
        output_placeholder: y_train
    }
    
    sess.run(train, feed_dict=train_dict)
    curr_error = sess.run(error, feed_dict=train_dict)
    if i % 10 == 0:
      accuracy = tf.equal(tf.argmax(output_placeholder, axis = 1), tf.argmax(output, axis=1))
      accuracy_final = tf.reduce_mean(tf.cast(accuracy, tf.float32))

      test_dict = {
          input_placeholder: x_test,
          output_placeholder: y_test
      }

      print("Error: {}".format(sess.run(error, feed_dict=test_dict)))

Error: 0.08789081126451492
Error: 0.08382114768028259
Error: 0.08069551736116409
Error: 0.07825588434934616
Error: 0.07632222771644592
Error: 0.0747675895690918
Error: 0.07350121438503265
Error: 0.07245724648237228
Error: 0.07158716022968292
Error: 0.07085475325584412
Error: 0.0702325776219368
Error: 0.06969963014125824
Error: 0.06923957914113998
Error: 0.06883963197469711
Error: 0.06848962604999542
Error: 0.06818144768476486
Error: 0.06790855526924133
Error: 0.06766558438539505
Error: 0.06744817644357681
Error: 0.06725272536277771


Non-PCA + All inputs

---

**1 hidden neuron = 0.0672**

5 hidden neuron = 0.0678

10 hidden neuron = 0.0739

15 hidden neuron = 0.0704

20 hidden neuron = 0.0697

---





> Non-PCA + Correlated Inputs



In [ ]:
dataset.corr()['depressed'].sort_values()

education_level         -0.097361
Married                 -0.063588
incoming_business       -0.028586
incoming_no_business    -0.026104
living_expenses         -0.024149
incoming_agricultural   -0.019415
labor_primary           -0.010302
Survey_id               -0.010181
sex                     -0.005700
gained_asset            -0.005111
farm_expenses           -0.005059
incoming_salary         -0.001813
Number_children          0.003406
lasting_investment       0.004459
incoming_own_farm        0.010019
other_expenses           0.011107
save_asset               0.011379
Ville_id                 0.027938
total_members            0.033125
durable_asset            0.038323
no_lasting_investmen     0.051973
Age                      0.100126
depressed                1.000000
Name: depressed, dtype: float64

In [ ]:
x_corr = dataset[['Age', 'Married', 'education_level', 'no_lasting_investmen']].values
y_corr = dataset[['depressed']].values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split (x_corr, y_corr, test_size = 0.2, random_state=0)

In [ ]:
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
y_train, y_test = y_train.astype('float32'), y_test.astype('float32')

In [ ]:
layers = {
    'input' : 4,
    'hidden' : 1,
    'output' : 1
}

input_hidden = {
    'weight': tf.Variable(tf.random_normal([layers['input'], layers['hidden']], seed=0)),
    'bias': tf.Variable(tf.random_normal([layers['hidden']], seed=0))
}

hidden_output = {
    'weight': tf.Variable(tf.random_normal([layers['hidden'], layers['output']], seed=0)),
    'bias': tf.Variable(tf.random_normal([layers['output']], seed=0))
}

In [ ]:
input_placeholder = tf.placeholder(tf.float32, [None, layers['input']])
output_placeholder = tf.placeholder(tf.float32, [None, layers['output']])

output = feed_forward(input_placeholder)
error = tf.reduce_mean(0.5 * (output_placeholder - output) ** 2)

learning_rate = 0.2
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

In [ ]:
epoch = 200

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  for i in range(epoch):
    train_dict = {
        input_placeholder: x_train,
        output_placeholder: y_train
    }
    
    sess.run(train, feed_dict=train_dict)
    curr_error = sess.run(error, feed_dict=train_dict)
    if i % 10 == 0:
      accuracy = tf.equal(tf.argmax(output_placeholder, axis = 1), tf.argmax(output, axis=1))
      accuracy_final = tf.reduce_mean(tf.cast(accuracy, tf.float32))

      test_dict = {
          input_placeholder: x_test,
          output_placeholder: y_test
      }

      print("Error: {}".format(sess.run(error, feed_dict=test_dict)))

Error: 0.14517712593078613
Error: 0.10989009588956833
Error: 0.09094343334436417
Error: 0.0810382217168808
Error: 0.07567958533763885
Error: 0.07261815667152405
Error: 0.07076913863420486
Error: 0.06959515810012817
Error: 0.0688168853521347
Error: 0.06828146427869797
Error: 0.0679011270403862
Error: 0.06762324273586273
Error: 0.0674150139093399
Error: 0.06725537031888962
Error: 0.06713033467531204
Error: 0.06703042984008789
Error: 0.06694908440113068
Error: 0.06688164919614792
Error: 0.06682475656270981
Error: 0.06677600741386414


Non-PCA + All inputs

---

1 hidden neuron = 0.0643

**5 hidden neuron = 0.0638**

10 hidden neuron = 0.0660

15 hidden neuron = 0.0659

20 hidden neuron = 0.0667

---





> PCA



In [ ]:
from sklearn.decomposition import PCA

In [ ]:
x_pca = dataset[['sex', 'Age', 'Married', 'Number_children', 'education_level', 'total_members', 'gained_asset', 'durable_asset', 'save_asset', 'living_expenses', 'other_expenses', 'incoming_salary', 'incoming_own_farm', 'incoming_business', 'incoming_no_business', 'incoming_agricultural', 'farm_expenses', 'labor_primary', 'lasting_investment', 'no_lasting_investmen']].values
y_pca = dataset[['depressed']].values

In [ ]:
scaler = StandardScaler().fit(x_pca)
x_pca = scaler.transform(x_pca)

In [ ]:
pca = PCA(n_components=4).fit(x_pca)
x_pca = pca.transform(x_pca)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split (x_pca, y_pca, test_size = 0.2, random_state=0)

In [ ]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
y_train, y_test = y_train.astype('float32'), y_test.astype('float32')

In [ ]:
layers = {
    'input' : 4,
    'hidden' : 1,
    'output' : 1
}

input_hidden = {
    'weight': tf.Variable(tf.random_normal([layers['input'], layers['hidden']], seed=0)),
    'bias': tf.Variable(tf.random_normal([layers['hidden']], seed=0))
}

hidden_output = {
    'weight': tf.Variable(tf.random_normal([layers['hidden'], layers['output']], seed=0)),
    'bias': tf.Variable(tf.random_normal([layers['output']], seed=0))
}

In [ ]:
input_placeholder = tf.placeholder(tf.float32, [None, layers['input']])
output_placeholder = tf.placeholder(tf.float32, [None, layers['output']])

output = feed_forward(input_placeholder)
error = tf.reduce_mean(0.5 * (output_placeholder - output) ** 2)

learning_rate = 0.2
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

In [ ]:
epoch = 200

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  for i in range(epoch):
    train_dict = {
        input_placeholder: x_train,
        output_placeholder: y_train
    }
    
    sess.run(train, feed_dict=train_dict)
    curr_error = sess.run(error, feed_dict=train_dict)
    if i % 10 == 0:
      accuracy = tf.equal(tf.argmax(output_placeholder, axis = 1), tf.argmax(output, axis=1))
      accuracy_final = tf.reduce_mean(tf.cast(accuracy, tf.float32))

      test_dict = {
          input_placeholder: x_test,
          output_placeholder: y_test
      }

      print("Error: {}".format(sess.run(error, feed_dict=test_dict)))

Error: 0.13025294244289398
Error: 0.11061280220746994
Error: 0.09583774954080582
Error: 0.08540740609169006
Error: 0.07846333831548691
Error: 0.07403206825256348
Error: 0.07126010209321976
Error: 0.06952782720327377
Error: 0.06843400746583939
Error: 0.0677327811717987
Error: 0.06727609783411026
Error: 0.06697438657283783
Error: 0.06677268445491791
Error: 0.06663656234741211
Error: 0.06654408574104309
Error: 0.06648095697164536
Error: 0.06643769145011902
Error: 0.0664079412817955
Error: 0.0663873627781868
Error: 0.06637294590473175


PCA

---

1 hidden neuron = 0.0652

**5 hidden neuron = 0.0639**

10 hidden neuron = 0.0668

15 hidden neuron = 0.0658

20 hidden neuron = 0.0663

---


Hasil paling ideal -> Non PCA + 4 Correlated Inputs dengan 5 hidden neuron

Error = 0.0638
